In [45]:
import numpy as np

import sklearn
from sklearn.datasets import make_swiss_roll, fetch_openml
from sklearn.decomposition import PCA, IncrementalPCA, KernelPCA
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_squared_error, accuracy_score
from sklearn.manifold import LocallyLinearEmbedding, t_sne

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.manifold.t_sne module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.manifold. Anything that cannot be imported from sklearn.manifold is now part of the private API.
  warnings.warn(message, FutureWarning)


In [2]:
x, y = make_swiss_roll(n_samples=300, noise=0.1, )

# Load the Mnist
mnist = fetch_openml(name='mnist_784', version=1)
x_mnist, y_mnist = mnist['data'], mnist['target']

#PCA (Principal Component Analysis)

## Principal Components

In [3]:
x_centered = x - x.mean(axis=0)
U, s, Vt = np.linalg.svd(x_centered)
component_1 = Vt.T[:, 0]
component_2 = Vt.T[:, 1]

## Projecting Down to d Dimensions

 - Projecting the training set down to d dimensions
 > $\textbf{X}_{d\text{-proj}} = \textbf{XW}_{d}$
 >
 > Where $\textbf{W}_d$ is defined as the matrix containing the first $d$ columns of the vector $\textbf{V}$

In [4]:
w2 = Vt.T[:, :2]
x2d = x_centered @ w2

## Using Scikit-Learn

In [5]:
pca = PCA(n_components=0.95)
x2d = pca.fit_transform(x)

## Explained Variance Ratio

In [6]:
pca.explained_variance_ratio_

array([0.43364394, 0.31992898, 0.24642708])

## PCA for Compression

In [7]:
pca_mnist = PCA(n_components=0.95)
x_mnist_reduced = pca_mnist.fit_transform(x_mnist)

x_mnist_reduced.shape

(70000, 154)

In [8]:
x_mnist_recovered = pca_mnist.inverse_transform(x_mnist_reduced)

- PCA inverse transformation, back to the original number of dimensions
> $\textbf{X}_{\text{recovered}} = \textbf{X}_{d\text{-proj}}\textbf{W}_{d}^{\intercal}$

## Randomized PCA




In [9]:
rnd_pca = PCA(n_components=154, svd_solver='randomized')
x_mnist_reduced = rnd_pca.fit_transform(x_mnist)

## Incremental PCA

In [10]:
n_batches = 100
inc_pca_mnist = IncrementalPCA(n_components=154)

for x_batch in np.array_split(x_mnist, n_batches):
  inc_pca_mnist.partial_fit(x_batch)

x_reduced_mnist = inc_pca_mnist.transform(x_mnist)

# Kernel PCA

In [11]:
rbf_pca = KernelPCA(n_components=2, kernel='rbf', gamma=0.04)
x_reduced = rbf_pca.fit_transform(x)

## Selecting a Kernel and Tuning Hyperparameters

In [12]:
clf = Pipeline([
  ('kpca', KernelPCA(n_components=2)),
  ('rnd_clf', RandomForestRegressor())
])

param_grid = [{
  'kpca__gamma' : np.linspace(0.03, 0.05, 10),
  'kpca__kernel' : ['rbf', 'sigmoid']
}]

grid_search = GridSearchCV(clf, param_grid, cv=3)
grid_search.fit(x, y)

GridSearchCV(cv=3, error_score=nan,
             estimator=Pipeline(memory=None,
                                steps=[('kpca',
                                        KernelPCA(alpha=1.0, coef0=1,
                                                  copy_X=True, degree=3,
                                                  eigen_solver='auto',
                                                  fit_inverse_transform=False,
                                                  gamma=None, kernel='linear',
                                                  kernel_params=None,
                                                  max_iter=None, n_components=2,
                                                  n_jobs=None,
                                                  random_state=None,
                                                  remove_zero_eig=False,
                                                  tol=0)),
                                       ('rnd_clf',
                                 

In [13]:
grid_search.best_params_

{'kpca__gamma': 0.05, 'kpca__kernel': 'rbf'}

In [14]:
# Enable the inverse reconstruction
rbf_pca = KernelPCA(n_components=2, kernel='rbf', gamma=0.0411, fit_inverse_transform=True)

x_reduced = rbf_pca.fit_transform(x)
x_preimage = rbf_pca.inverse_transform(x_reduced)

In [15]:
mean_squared_error(x, x_preimage)

30.338161016347982

# LLE (Local Linear Embedding)

In [16]:
lle = LocallyLinearEmbedding(n_components=2, n_neighbors=10)
x_reduced = lle.fit_transform(x)

- LLE step one: linearly modeling local relationships
> $\hat{\textbf{W}} = \underset{\textbf{W}}{\text{argmin}} \sum\limits_{i=1}^{m}{(\textbf{x}^{(i)} - \sum\limits_{j=1}^{m}{w_{i,j}\textbf{x}^{(j)}})}$
>
> $\text{subject to} \begin{cases}{w_{i,j}=0 \quad \text{if } \textbf{x}^{(j)} \text{ is not one of the } k \text{ c.n. of }\textbf{x}^{(i)}} \\ {\sum\limits_{j=1}^{m}{w_{i,j}=1} \quad \text{ for } i=1, 2,..., m}\end{cases}$

- LLE second step: reducing dimensionality while preserving relationships
> $\hat{\textbf{Z}} = \underset{\textbf{Z}}{\text{argmin}}\sum\limits_{i=1}^{m}{(\textbf{z}^{(i)} - \sum\limits_{j=1}^{m}{ \hat{w}_{i,j} \textbf{z}^{(j)}}) ^ {2}}$

# Exercises

1. What are the main motivations for reducing a dataset’s dimensionality? What are the main drawbacks?

> The main motivations for reducing a dataset's dimensionality are to speed up the training, to improve the visualization of the data in 3D or 2D, and finally compressing the data when the data set is too large.
>
> The main drawbacks are:     
>- Some information is lost, possibly degrading the performance of subsequent training algorithms.
> - It can be computationally intensive.
> - It adds some complexity to your Machine Learning pipelines.
> - Transformed features are often hard to interpret.

2. What is the curse of dimensionality?

> The curse of dimensionality refers to the fact that many problems that do not exist in low-dimensional space arise in high-dimensional space. In Machine Learning, one common manifestation is the fact that randomly sampled highdimensional vectors are generally very sparse, increasing the risk of overfitting and making it very difficult to identify patterns in the data without having plenty of training data.


3. Once a dataset’s dimensionality has been reduced, is it possible to reverse the operation? If so, how? If not, why?

> Once we have a reduced dataset, it is possible to reverse the operation by multiplying the reduced vector by the transposed of the vector W (with d principal components). However, this is not always possible, and most of the time there will be information that is lost in the process.

4. Can PCA be used to reduce the dimensionality of a highly nonlinear dataset?

PCA can be used to significantly reduce the dimensionality of most datasets, even if they are highly nonlinear, because it can at least get rid of useless dimensions. However, if there are no useless dimensions—as in a Swiss roll dataset—then reducing dimensionality with PCA will lose too much information

5. Suppose you perform PCA on a 1,000-dimensional dataset, setting the explained variance ratio to 95%. How many dimensions will the resulting dataset have?

> The resulting dataset will have $d$ dimensions, where $d\le n$. The important is that these $d$ dimensions explain the 95% of the variation of the data

6. In what cases would you use vanilla PCA, Incremental PCA, Randomized PCA, or Kernel PCA?

> - Vanilla PCA is a good starting point when the size of the size of the data is very large and $d$ is more than $85%$ of $n$.
> - Incremental PCA is a good default when the size of the data is very large and may not fit in memory. This allows to load the data by chunks.
> - Randomized PCA helps to speed the dimensionality reduction when $m$ is less than $80%$ than $n$.
> - Kernel PCA is good when the data is non linear and the application of a kernel may help.

7. How can you evaluate the performance of a dimensionality reduction algorithm on your dataset?

> - Since PCA is an unsupervised algorithm, there is no metric to evaluate it. However, it is possible to create a pipeline and evaluate the result of the supervised algorithm that will use the reduced data. Measuring the performance of the supervised algorithm is a good indicator on how the PCA performs.

8. Does it make any sense to chain two different dimensionality reduction algorithms?

>It can absolutely make sense to chain two different dimensionality reduction
algorithms. A common example is using PCA to quickly get rid of a large num‐
ber of useless dimensions, then applying another much slower dimensionality
reduction algorithm, such as LLE. This two-step approach will likely yield the
same performance as using LLE only, but in a fraction of the time.

9. Load the MNIST dataset (introduced in Chapter 3) and split it into a training set and a test set (take the first 60,000 instances for training, and the remaining 10,000 for testing). Train a Random Forest classifier on the dataset and time how long it takes, then evaluate the resulting model on the test set. Next, use PCA to reduce the dataset’s dimensionality, with an explained variance ratio of 95%. Train a new Random Forest classifier on the reduced dataset and see how long it
takes. Was training much faster? Next, evaluate the classifier on the test set. How does it compare to the previous classifier?


In [22]:
# Load the Mnist
mnist = fetch_openml(name='mnist_784', version=1)
x_mnist, y_mnist = mnist['data'], mnist['target']

In [23]:
x_train_mnist, x_test_mnist = x_mnist[:60000], x_mnist[60000:]
y_train_mnist, y_test_mnist = y_mnist[:60000], y_mnist[60000:]

In [35]:
rnd_clf = RandomForestClassifier()

%time rnd_clf.fit(x_train_mnist, y_train_mnist)

CPU times: user 48.6 s, sys: 43.1 ms, total: 48.6 s
Wall time: 48.3 s


RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [27]:
y_pred_mnist = rnd_clf.predict(x_test_mnist)

print(f'The accuracy of the Random Forest with the entire data is {accuracy_score(y_test_mnist, y_pred_mnist)}')

The accuracy of the Random Forest with the entire data is 0.9699


In [28]:
pca_mnist = PCA(n_components=0.95)

x_mnist_reduced = pca_mnist.fit_transform(x_train_mnist)

CPU times: user 2min 5s, sys: 39.6 ms, total: 2min 5s
Wall time: 2min 4s


RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [37]:
reducing_pipeline = Pipeline([
  ('pca_mnist', PCA(n_components=0.95)),
  ('rnd_clf', RandomForestClassifier())
])

In [43]:
%time reducing_pipeline.fit(x_train_mnist, y_train_mnist)

CPU times: user 2min 22s, sys: 1.53 s, total: 2min 23s
Wall time: 2min 13s


Pipeline(memory=None,
         steps=[('pca_mnist',
                 PCA(copy=True, iterated_power='auto', n_components=0.95,
                     random_state=None, svd_solver='auto', tol=0.0,
                     whiten=False)),
                ('rnd_clf',
                 RandomForestClassifier(bootstrap=True, ccp_alpha=0.0,
                                        class_weight=None, criterion='gini',
                                        max_depth=None, max_features='auto',
                                        max_leaf_nodes=None, max_samples=None,
                                        min_impurity_decrease=0.0,
                                        min_impurity_split=None,
                                        min_samples_leaf=1, min_samples_split=2,
                                        min_weight_fraction_leaf=0.0,
                                        n_estimators=100, n_jobs=None,
                                        oob_score=False, random_state=None,
       

In [42]:
y_pred_reduced_mnist = reducing_pipeline.predict(x_test_mnist)

print(f'The accuracy of the Random Forest with the reduced data is {accuracy_score(y_test_mnist, y_pred_reduced_mnist)}')

The accuracy of the Random Forest with the reduced data is 0.9489


10. Use t-SNE to reduce the MNIST dataset down to two dimensions and plot the result using Matplotlib. You can use a scatterplot using 10 different colors to represent each image’s target class. Alternatively, you can replace each dot in the scatterplot with the corresponding instance’s class (a digit from 0 to 9), or even plot scaled-down versions of the digit images themselves (if you plot all digits,
the visualization will be too cluttered, so you should either draw a random sample or plot an instance only if no other instance has already been plotted at a close distance). You should get a nice visualization with well-separated clusters of digits. Try using other dimensionality reduction algorithms such as PCA, LLE, or MDS and compare the resulting visualizations.

In [48]:
help(t_sne.Pep562)

Help on class Pep562 in module sklearn.externals._pep562:

class Pep562(builtins.object)
 |  Pep562(name)
 |  
 |  Backport of PEP 562 <https://pypi.org/search/?q=pep562>.
 |  
 |  Wraps the module in a class that exposes the mechanics to override `__dir__` and `__getattr__`.
 |  The given module will be searched for overrides of `__dir__` and `__getattr__` and use them when needed.
 |  
 |  Methods defined here:
 |  
 |  __dir__(self)
 |      Return the overridden `dir` if one was provided, else apply `dir` to the module.
 |  
 |  __getattr__(self, name)
 |      Attempt to retrieve the attribute from the module, and if missing, use the overridden function if present.
 |  
 |  __init__(self, name)
 |      Acquire `__getattr__` and `__dir__`, but only replace module for versions less than Python 3.7.
 |  
 |  ----------------------------------------------------------------------
 |  Data descriptors defined here:
 |  
 |  __dict__
 |      dictionary for instance variables (if defined)
 